In [ ]:
%load_ext autoreload
%autoreload 2
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="MIG-7d67cf82-0c6b-5aee-99ce-cab54f47f0b6" 
from hydra.utils import instantiate
import yaml
import torch
import numpy as np
import torch
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from PIL import Image
device = "cuda:0"

In [ ]:
1

In [ ]:
1

In [ ]:
from src.data.get_datamodules import get_pcna_data
from cyto_dl.models.utils.mlflow import load_model_from_checkpoint
from src.models.save_embeddings import get_pc_loss

def get_data_and_models():
    data = get_pcna_data()
    data_list = [data[0], data[0], data[1], data[-2], data[-2]]

    run_ids = [
        "6bb8003c781e4661914a768763a352a3",
        "68157cde6680432292c3f5b959394bec",
        "e41f0bbf26944ba9b9e4541e87bf42f0",
        "358cef46219c435ab1b16d2a06dd4436",
    ]
    run_names = ["2048_ed_dgcnn", "2048_int_ed_vndgcnn", "classical_image", "so2_image"]
    all_models = []
    for i in run_ids:
        all_models.append(
            load_model_from_checkpoint(
                "https://mlflow.a100.int.allencell.org",
                i,
                path="checkpoints/val/loss/best.ckpt",
                strict=False,
            )
        )
    return data_list, all_models, run_names

In [ ]:
data_list, all_models, run_names = get_data_and_models()

In [ ]:
run_names

In [ ]:

# image models first
all_models.reverse()
data_list.reverse()
run_names.reverse()

In [ ]:
run_names

In [ ]:
from src.models.save_embeddings import save_embeddings

debug = False
path = "./embeddings"
save_embeddings(path, data_list, all_models, run_names, debug)

In [ ]:
from src.models.save_embeddings import save_emissions

debug = False
path = "./emissions"
save_emissions(path, data_list, all_models, run_names, 5, debug)

In [ ]:
run_names

In [ ]:
from src.models.compute_features import compute_features

save_folder = './pcna_features/'
keys = ['image', 'image', 'pcloud', 'pcloud']
compute_features(
    dataset='pcna',
    save_folder=save_folder,
    data_list=data_list,
    all_models=all_models,
    run_names=run_names,
    keys=keys,
    device="cuda:0",
    max_embed_dim=256,
    class_label='cell_stage_fine',
    num_evolve_samples=2,
    squeeze_2d=False,
)